In [1]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        'XXXXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

In [2]:
query_1 = """ 
    
select distinct YEAR(VisitStartDateTime) as year, MONTH(VisitStartDateTime) as month, DAY(VisitStartDateTime) as day, sessionuid, sceneuid, VisitStartDateTime, locationId, SceneType, SubSceneType, TipoMueble, FabMueble, productid from dbo.as_ir_report    
where YEAR(VisitStartDateTime) = '2021' and SubSceneType = 'Exhibiciones Especiales' and FabMueble in ('Exhibición Coca-Cola', 'Exhibición Competencia')

        """

In [3]:
df = pd.DataFrame(get_query(query_1))
#df = df.sort_values(by="VisitStartDateTime",ascending=False)
#df

In [4]:
df['date_format']=pd.to_datetime(df[['year','month','day']])
#df.info()

In [5]:
df=df.rename(columns={"sessionuid": "SessionUid"})

In [6]:
df.duplicated(subset='SessionUid', keep='first').sum()

3609284

In [7]:
df=df.drop_duplicates(subset=['SessionUid'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13223 entries, 0 to 3355017
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   year                13223 non-null  int64         
 1   month               13223 non-null  int64         
 2   day                 13223 non-null  int64         
 3   SessionUid          13223 non-null  object        
 4   sceneuid            13223 non-null  object        
 5   VisitStartDateTime  13223 non-null  datetime64[ns]
 6   locationId          13223 non-null  int64         
 7   SceneType           13223 non-null  object        
 8   SubSceneType        13223 non-null  object        
 9   TipoMueble          13223 non-null  object        
 10  FabMueble           13223 non-null  object        
 11  productid           13223 non-null  int64         
 12  date_format         13223 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(5), object(6)
me

In [8]:

query_2 = """ 
    
select distinct SessionUid, LocationId, UserId, DateId, ChainDsc, Format, Clasification, ICE, exh_ko, exh_share, is_reported from dbo.r_autoservicios_v3    


        """

In [9]:
df2 = pd.DataFrame(get_query(query_2))
#df = df.sort_values(by="date_format",ascending=True)
#df2

In [10]:
dff = pd.merge(df2, df, on='SessionUid', how='inner')
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13223 entries, 0 to 13222
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SessionUid          13223 non-null  object        
 1   LocationId          13223 non-null  int64         
 2   UserId              13223 non-null  int64         
 3   DateId              13223 non-null  object        
 4   ChainDsc            13205 non-null  object        
 5   Format              13223 non-null  object        
 6   Clasification       13205 non-null  object        
 7   ICE                 13223 non-null  float64       
 8   exh_ko              13223 non-null  int64         
 9   exh_share           13223 non-null  int64         
 10  is_reported         13223 non-null  int64         
 11  year                13223 non-null  int64         
 12  month               13223 non-null  int64         
 13  day                 13223 non-null  int64     

In [11]:
table = pd.pivot_table(dff, values=['exh_ko','exh_share'], index=['UserId', 'LocationId', 'is_reported', 'date_format', 'month', 'day', 'SessionUid'],
                     fill_value=0)
table=pd.DataFrame(table)
table

exh_ko  \
UserId LocationId is_reported date_format month day SessionUid                                     
39272  9753598    0           2021-07-23  7     23  f282ba74-72af-4417-9892-762e0b4ae955       0   
43976  6473384    0           2021-06-02  6     2   436f68af-a9f0-4ba6-ba59-5aadf701b2b9       3   
                              2021-06-24  6     24  4bd0947a-e32a-4116-9128-115508b65a53       2   
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd      20   
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b      13   
...                                                                                          ...   
57678  6585311    1           2021-08-20  8     20  d8c09c22-191c-431f-a116-bb4129ed7cb1      10   
       6587353    1           2021-09-15  9     15  07a36d1c-4bc1-40dd-89c9-b91551da28ae      13   
       7277804    1           2021-09-08  9     8   0b569439-e7b6-430f-97d9-a0c6e285ec57      13   
       7277813    1           2021-09-08  9     8   d22b8943-b627-4908-9a10-3cd711343a6f      13   
       9813408    1           2021-09-13  9     13  d99b571c-4316-4029-9296-7084daa47921      10   

                                                                                          exh_share  
UserId LocationId is_reported date_format month day SessionUid                                       
39272  9753598    0           2021-07-23  7     23  f282ba74-72af-4417-9892-762e0b4ae955          1  
43976  6473384    0           2021-06-02  6     2   436f68af-a9f0-4ba6-ba59-5aadf701b2b9          7  
                              2021-06-24  6     24  4bd0947a-e32a-4116-9128-115508b65a53          2  
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd         28  
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b         13  
...                                                                                             ...  
57678  6585311    1           2021-08-20  8     20  d8c09c22-191c-431f-a116-bb4129ed7cb1         51  
       6587353    1           2021-09-15  9     15  07a36d1c-4bc1-40dd-89c9-b91551da28ae         35  
       7277804    1           2021-09-08  9     8   0b569439-e7b6-430f-97d9-a0c6e285ec57         45  
       7277813    1           2021-09-08  9     8   d22b8943-b627-4908-9a10-3cd711343a6f         31  
       9813408    1           2021-09-13  9     13  d99b571c-4316-4029-9296-7084daa47921         32  

[13223 rows x 2 columns]

In [12]:
table = table.reset_index()
table

,UserId,LocationId,is_reported,date_format,month,day,SessionUid,exh_ko,exh_share
0,39272,9753598,0,2021-07-23,7,23,f282ba74-72af-4417-9892-762e0b4ae955,0,1
1,43976,6473384,0,2021-06-02,6,2,436f68af-a9f0-4ba6-ba59-5aadf701b2b9,3,7
2,43976,6473384,0,2021-06-24,6,24,4bd0947a-e32a-4116-9128-115508b65a53,2,2
3,43978,6473384,0,2021-06-01,6,1,1fffc695-3619-49c5-b01b-d4510c2ec5fd,20,28
4,43978,6473384,0,2021-06-17,6,17,f5caea7c-a7f2-4c14-982e-8094c25eb31b,13,13
...,...,...,...,...,...,...,...,...,...
13218,57678,6585311,1,2021-08-20,8,20,d8c09c22-191c-431f-a116-bb4129ed7cb1,10,51
13219,57678,6587353,1,2021-09-15,9,15,07a36d1c-4bc1-40dd-89c9-b91551da28ae,13,35
13220,57678,7277804,1,2021-09-08,9,8,0b569439-e7b6-430f-97d9-a0c6e285ec57,13,45
13221,57678,7277813,1,2021-09-08,9,8,d22b8943-b627-4908-9a10-3cd711343a6f,13,31


In [13]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13223 entries, 0 to 13222
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   UserId       13223 non-null  int64         
 1   LocationId   13223 non-null  int64         
 2   is_reported  13223 non-null  int64         
 3   date_format  13223 non-null  datetime64[ns]
 4   month        13223 non-null  int64         
 5   day          13223 non-null  int64         
 6   SessionUid   13223 non-null  object        
 7   exh_ko       13223 non-null  int64         
 8   exh_share    13223 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 929.9+ KB


In [14]:
table['exh_comp']=table['exh_share']-table['exh_ko']
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13223 entries, 0 to 13222
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   UserId       13223 non-null  int64         
 1   LocationId   13223 non-null  int64         
 2   is_reported  13223 non-null  int64         
 3   date_format  13223 non-null  datetime64[ns]
 4   month        13223 non-null  int64         
 5   day          13223 non-null  int64         
 6   SessionUid   13223 non-null  object        
 7   exh_ko       13223 non-null  int64         
 8   exh_share    13223 non-null  int64         
 9   exh_comp     13223 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 1.0+ MB


In [15]:
moni=table.groupby(['UserId','LocationId', 'SessionUid', 'is_reported', 'date_format', 'month', 'day'])['exh_ko', 'exh_comp'].sum()

moni  

<ipython-input-15-fb75ca5b84b0>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  moni=table.groupby(['UserId','LocationId', 'SessionUid', 'is_reported', 'date_format', 'month', 'day'])['exh_ko', 'exh_comp'].sum()


exh_ko  \
UserId LocationId SessionUid                           is_reported date_format month day           
39272  9753598    f282ba74-72af-4417-9892-762e0b4ae955 0           2021-07-23  7     23        0   
43976  6473384    436f68af-a9f0-4ba6-ba59-5aadf701b2b9 0           2021-06-02  6     2         3   
                  4bd0947a-e32a-4116-9128-115508b65a53 0           2021-06-24  6     24        2   
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1        20   
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17       13   
...                                                                                          ...   
57678  6585311    d8c09c22-191c-431f-a116-bb4129ed7cb1 1           2021-08-20  8     20       10   
       6587353    07a36d1c-4bc1-40dd-89c9-b91551da28ae 1           2021-09-15  9     15       13   
       7277804    0b569439-e7b6-430f-97d9-a0c6e285ec57 1           2021-09-08  9     8        13   
       7277813    d22b8943-b627-4908-9a10-3cd711343a6f 1           2021-09-08  9     8        13   
       9813408    d99b571c-4316-4029-9296-7084daa47921 1           2021-09-13  9     13       10   

                                                                                          exh_comp  
UserId LocationId SessionUid                           is_reported date_format month day            
39272  9753598    f282ba74-72af-4417-9892-762e0b4ae955 0           2021-07-23  7     23          1  
43976  6473384    436f68af-a9f0-4ba6-ba59-5aadf701b2b9 0           2021-06-02  6     2           4  
                  4bd0947a-e32a-4116-9128-115508b65a53 0           2021-06-24  6     24          0  
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1           8  
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17          0  
...                                                                                            ...  
57678  6585311    d8c09c22-191c-431f-a116-bb4129ed7cb1 1           2021-08-20  8     20         41  
       6587353    07a36d1c-4bc1-40dd-89c9-b91551da28ae 1           2021-09-15  9     15         22  
       7277804    0b569439-e7b6-430f-97d9-a0c6e285ec57 1           2021-09-08  9     8          32  
       7277813    d22b8943-b627-4908-9a10-3cd711343a6f 1           2021-09-08  9     8          18  
       9813408    d99b571c-4316-4029-9296-7084daa47921 1           2021-09-13  9     13         22  

[13223 rows x 2 columns]

In [16]:
table2 = moni.reset_index()
table2

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,exh_ko,exh_comp
0,39272,9753598,f282ba74-72af-4417-9892-762e0b4ae955,0,2021-07-23,7,23,0,1
1,43976,6473384,436f68af-a9f0-4ba6-ba59-5aadf701b2b9,0,2021-06-02,6,2,3,4
2,43976,6473384,4bd0947a-e32a-4116-9128-115508b65a53,0,2021-06-24,6,24,2,0
3,43978,6473384,1fffc695-3619-49c5-b01b-d4510c2ec5fd,0,2021-06-01,6,1,20,8
4,43978,6473384,f5caea7c-a7f2-4c14-982e-8094c25eb31b,0,2021-06-17,6,17,13,0
...,...,...,...,...,...,...,...,...,...
13218,57678,6585311,d8c09c22-191c-431f-a116-bb4129ed7cb1,1,2021-08-20,8,20,10,41
13219,57678,6587353,07a36d1c-4bc1-40dd-89c9-b91551da28ae,1,2021-09-15,9,15,13,22
13220,57678,7277804,0b569439-e7b6-430f-97d9-a0c6e285ec57,1,2021-09-08,9,8,13,32
13221,57678,7277813,d22b8943-b627-4908-9a10-3cd711343a6f,1,2021-09-08,9,8,13,18


In [17]:
table2=table2.sort_values(['LocationId', 'date_format'])
table2

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,exh_ko,exh_comp
11026,52049,6188351,6fbb054a-4bb4-4660-b055-9be64eb62409,1,2021-05-21,5,21,19,52
3619,45847,6188351,7dbcf5ac-dfb7-4a2c-9c74-62ee7957352e,1,2021-06-03,6,3,19,45
3618,45847,6188351,549e6699-f573-4a84-ba02-53f02693fe43,1,2021-07-06,7,6,16,53
11027,52049,6188351,a38e7b33-6b0f-42e3-a8bc-7991cb5fb7e0,1,2021-08-14,8,14,22,46
11025,52049,6188351,2c000823-ce4b-4d38-a733-37a9e06b29c7,1,2021-09-09,9,9,19,51
...,...,...,...,...,...,...,...,...,...
1022,45793,10051235,4e2bf448-b164-4323-969c-5897eeee33ca,1,2021-08-07,8,7,15,10
13195,57355,10051235,5da0103c-71bc-4cba-b706-d5af482cb097,1,2021-09-04,9,4,13,13
552,45789,10068942,16f38349-86f4-445e-b820-b39ea61a3a23,1,2021-07-14,7,14,21,12
554,45789,10068942,c651604d-d15e-43ef-8023-56b3ddb15eb9,1,2021-08-07,8,7,22,5


In [18]:
table2["Coca-Cola"] = table2["exh_ko"].shift(1)
table2["Competencia"] = table2["exh_comp"].shift(1)
table2.dtypes

UserId                  int64
LocationId              int64
SessionUid             object
is_reported             int64
date_format    datetime64[ns]
month                   int64
day                     int64
exh_ko                  int64
exh_comp                int64
Coca-Cola             float64
Competencia           float64
dtype: object

In [19]:
table2 = table2.fillna(0)

In [85]:
table2 = table2.astype({"Coca-Cola": int, "Competencia": int, "LocationId": object})

In [86]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6334 entries, 5590 to 1
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   UserId       6334 non-null   int64         
 1   LocationId   6334 non-null   object        
 2   SessionUid   6334 non-null   object        
 3   is_reported  6334 non-null   int64         
 4   date_format  6334 non-null   datetime64[ns]
 5   month        6334 non-null   int64         
 6   day          6334 non-null   int64         
 7   exh_ko       6334 non-null   int64         
 8   exh_comp     6334 non-null   int64         
 9   Coca-Cola    6334 non-null   int32         
 10  Competencia  6334 non-null   int32         
dtypes: datetime64[ns](1), int32(2), int64(6), object(2)
memory usage: 544.3+ KB


In [87]:
table2["LocationIdChanged"] = table2["LocationId"].shift(1, fill_value=table2["LocationId"].head(1)) != table2["LocationId"]
table2.dtypes

UserId                        int64
LocationId                   object
SessionUid                   object
is_reported                   int64
date_format          datetime64[ns]
month                         int64
day                           int64
exh_ko                        int64
exh_comp                      int64
Coca-Cola                     int32
Competencia                   int32
LocationIdChanged              bool
dtype: object

In [90]:
table2["LocationIdChanged"] = table2["LocationIdChanged"].astype(object)
table2.dtypes

UserId                        int64
LocationId                   object
SessionUid                   object
is_reported                   int64
date_format          datetime64[ns]
month                         int64
day                           int64
exh_ko                        int64
exh_comp                      int64
Coca-Cola                     int32
Competencia                   int32
LocationIdChanged            object
dtype: object

In [91]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_CC = 'change'
              
          else: 
              Flag_CC = table2["exh_ko"] - table2["Coca-Cola"]

              table2['Flag_CC']=Flag_CC
              
      #print(Flag_CC)   

In [92]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Comp = 'change'
              
          else: 
              Flag_Comp = table2["exh_comp"] - table2["Competencia"]

              table2['Flag_Comp']=Flag_Comp
              
      #print(Flag_Comp)  

In [93]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6334 entries, 5590 to 1
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   UserId             6334 non-null   int64         
 1   LocationId         6334 non-null   object        
 2   SessionUid         6334 non-null   object        
 3   is_reported        6334 non-null   int64         
 4   date_format        6334 non-null   datetime64[ns]
 5   month              6334 non-null   int64         
 6   day                6334 non-null   int64         
 7   exh_ko             6334 non-null   int64         
 8   exh_comp           6334 non-null   int64         
 9   Coca-Cola          6334 non-null   int32         
 10  Competencia        6334 non-null   int32         
 11  LocationIdChanged  6334 non-null   object        
 12  Flag_CC            6334 non-null   int64         
 13  Flag_Comp          6334 non-null   int64         
dtypes: datet

In [96]:
table_final = table2[['UserId','LocationId','SessionUid', 'is_reported', 'date_format', 'month', 'day', 'LocationIdChanged', 'exh_ko', 'Flag_CC', 'exh_comp', 'Flag_Comp']]
table_final

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,LocationIdChanged,exh_ko,Flag_CC,exh_comp,Flag_Comp
5590,52049,6188351,6fbb054a-4bb4-4660-b055-9be64eb62409,0,2021-05-21,5,21,False,19,19,52,52
5198,47713,6188352,224b5b6e-830f-4f4a-beac-a71ff9cfe6b0,1,2021-01-14,1,14,True,29,10,28,-24
5199,47713,6188352,b8fa381a-239f-4163-ae89-45242b2ecb3c,1,2021-02-15,2,15,False,29,0,46,18
5197,47713,6188352,1a2a622a-9f38-4df9-9c8f-964c8b618f53,1,2021-03-04,3,4,False,37,8,34,-12
5200,47713,6188352,e24fc858-15e6-4758-a5ab-9a5c8e2b8907,1,2021-04-02,4,2,False,40,3,48,14
...,...,...,...,...,...,...,...,...,...,...,...,...
45,45788,7290464,fc406fc3-d60e-49fd-bf62-dad1d6d514f5,1,2021-05-13,5,13,False,5,-1,16,-1
5022,45936,9714248,d59b2af0-27ee-4984-a005-e17843145a9b,0,2021-05-20,5,20,True,8,3,48,32
5023,45936,9714249,c0ab447c-0653-47e7-aad9-317f9401f943,0,2021-05-20,5,20,True,7,-1,9,-39
0,45053,9753598,29f9987f-7701-4389-8b2b-f6163b1f781f,0,2021-05-08,5,8,True,7,0,10,1


In [97]:
table_final = table_final.fillna(0)

In [ ]:
print("flags-tt - Borrando los datos de la tabla")
query_clear_Exh_as = """ 
    
DELETE FROM reportes.Exhibidores_as

        """

In [ ]:
def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        '8khHAM=B=q$ZGr6N',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

eng = get_engine()
conn = eng.connect()
conn.execute(query_clear_Exh_as)
conn.close()
#get_query(query_clear_Exh_as_cc_comp)


print("flags-as - Llenando la tabla")

In [98]:
engine = get_engine()
table_final.to_sql('Exhibidores_as', con=engine, schema='reportes', if_exists='append', index=False)